In [1]:
import ast
import pandas as pd
import numpy as np
import os
from time import ctime
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows=300
%matplotlib inline

In [ ]:
%%time
fl=pd.read_csv('./info.label.train.csv', chunksize=10000, low_memory=False)
df = pd.concat([i for i in fl], ignore_index=True)
print df.shape

In [5]:
df.iloc[100:101,:].transpose()

,100
user_id,733565
indexdata_,jepps.html
about.avatar,/img/unknown.png
about.avatar_cdn,//cdn6.f-cdn.com/img/unknown.png
about.avatar_large,/img/unknown.png
about.avatar_large_cdn,//cdn6.f-cdn.com/img/unknown.png
about.chosen_role,2
about.company,NaN
about.display_name,JEPPS
about.hourly_rate,None


In [11]:
stat = lambda x: df[x].value_counts(dropna=False).head(5)

In [63]:
selected = ['label','user_id', 'about.company','about.hourly_rate','about.profile_description',\
            'about.registration_date_localized','about.recommendations','country.name',\
           'timezone','skills_LEN','resume.experience_LEN','resume.education_LEN','resume.certifications_LEN',\
           'job_history.job_counts_LEN','phone_verified','payment_verified','facebook_connected',\
           'summary.reviews','summary.on_time','summary.on_budget','entire_history.incomplete',\
           'entire_history.reviews','earnings_score','about.display_name','about.public_name',\
           'public_firstname']
len(selected)

26

### clean individual

In [21]:
df['about.company'] = df['about.company'].fillna('None')
df['about.company'] = df['about.company'].str.replace('none','None').replace('n/a','None').replace('-','None')

In [64]:
df_save = df[selected]
df_save.shape

(102514, 26)

In [67]:
df_save['label'] = df_save['label'].map({1:'Male',0:'Female'})

/home/shj16110/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [78]:
!ls -alhts|grep info

 43M -rw-r--r--  1 shj16110 Domain_Users  43M Dec 28 10:14 info.to.hive.csv
 64K -rw-r--r--  1 shj16110 Domain_Users  28K Dec 28 10:14 info_to_hive.ipynb
 43M -rw-r--r--  1 shj16110 Domain_Users  43M Dec 28 10:05 info.to.hive
330M -rw-r--r--  1 shj16110 Domain_Users 330M Dec 28 09:35 info.label.train.csv


In [90]:
from time import strftime, strptime

https://docs.python.org/2/library/datetime.html

In [95]:
df['about.registration_date_localized'][0]

'March 24, 2009'

In [105]:
sample = strptime(df['about.registration_date_localized'][0], '%B %d, %Y')
sample

time.struct_time(tm_year=2009, tm_mon=3, tm_mday=24, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=1, tm_yday=83, tm_isdst=-1)

In [110]:
'{}-{}-{}'.format(sample.tm_year, sample.tm_mon, sample.tm_mday)

'2009-3-24'

In [115]:
datetime.strptime(df['about.registration_date_localized'][0], '%B %d, %Y').strftime('%Y-%m-%d')

'2009-03-24'

In [119]:
from datetime import datetime
def helper_time(x):
    return datetime.strptime(x, '%B %d, %Y').strftime('%Y-%m-%d')

In [122]:
df_save['registration_date'] = df_save['about.registration_date_localized'].apply(helper_time)

/home/shj16110/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [126]:
df_save = df_save.drop('about.registration_date_localized', axis=1)
df_save.shape

(102514, 26)

In [14]:
%%time
df_save = pd.read_csv('info.to.hive.csv')
print df_save.shape

(102514, 26)
CPU times: user 489 ms, sys: 28 ms, total: 517 ms
Wall time: 660 ms


In [20]:
for i in df_save.select_dtypes(include=['float64'])[:-1]:
    df_save[i]=df_save[i].astype(int)

In [21]:
df_save.dtypes

label                         object
user_id                        int64
about.company                 object
about.hourly_rate             object
about.profile_description     object
about.recommendations          int64
country.name                  object
timezone                      object
skills_LEN                     int64
resume.experience_LEN          int64
resume.education_LEN           int64
resume.certifications_LEN      int64
job_history.job_counts_LEN     int64
phone_verified                  bool
payment_verified                bool
facebook_connected              bool
summary.reviews                int64
summary.on_time               object
summary.on_budget             object
entire_history.incomplete      int64
entire_history.reviews         int64
earnings_score                 int64
about.display_name            object
about.public_name             object
public_firstname              object
registration_date             object
dtype: object

In [22]:
%%time
df_save.to_csv('info.to.hive.csv', index=False)

CPU times: user 814 ms, sys: 7 ms, total: 821 ms
Wall time: 2.45 s


In [128]:
%%time
df_save = pd.read_csv('./info.to.hive.csv')

CPU times: user 491 ms, sys: 30 ms, total: 521 ms
Wall time: 518 ms


In [129]:
df_save.iloc[0:1,:].transpose()

,0
label,Male
user_id,1018449
about.company,itsol
about.hourly_rate,60
about.profile_description,NaN
about.recommendations,0
country.name,United Kingdom
timezone,Europe/London
skills_LEN,15
resume.experience_LEN,1


In [130]:
df_save['about.profile_description'] = df_save['about.profile_description'].str.replace('\n','    ').str.replace('\t','  ').str.replace('\\r','    ')

In [131]:
df_save['about.profile_description'][24]

"We are a Pacific Northwest based software company that works in the area of internet security.                As a very experience buyer, and we believe in treating the providers with the respect they deserve (some don't deserve as much respect as other :-| )                But at the end of the day you cannot expect the provider to read your mind, so you have to know what you want and recognize good work when you see it.                We have several very senior developers, but constantly need extra manpower for very specific sub-tasks.                All our projects are small and manageable, and you should easily be able to deliver within 2 weeks if you work full time.                Initial Projects will usually be less than $500                We understand that many service providers are small independent people who try to make a little extra by working evening or night, and we would like to encourage these hardworking individuals to make a bid.                We like the perso

In [132]:
n=0
for i in df_save.columns[:-1]:
    n+=1
    print str(n)+'_'+i.replace('.','_'), \
    str(df_save[i].dtypes).replace('object','STRING').replace('int64','INT').replace('float64','FLOAT')\
    .replace('bool','BOOLEAN')+','
print "26_registration_date DATE"

1_label STRING,
2_user_id INT,
3_about_company STRING,
4_about_hourly_rate STRING,
5_about_profile_description STRING,
6_about_recommendations INT,
7_country_name STRING,
8_timezone STRING,
9_skills_LEN INT,
10_resume_experience_LEN INT,
11_resume_education_LEN INT,
12_resume_certifications_LEN INT,
13_job_history_job_counts_LEN INT,
14_phone_verified BOOLEAN,
15_payment_verified BOOLEAN,
16_facebook_connected BOOLEAN,
17_summary_reviews INT,
18_summary_on_time STRING,
19_summary_on_budget STRING,
20_entire_history_incomplete INT,
21_entire_history_reviews INT,
22_earnings_score INT,
23_about_display_name STRING,
24_about_public_name STRING,
25_public_firstname STRING,
26_registration_date DATE


https://stackoverflow.com/questions/15751999/hive-external-table-skip-first-row

In [133]:
pre="""
CREATE EXTERNAL TABLE IF NOT EXISTS info(
"""

In [142]:
post = """
)
COMMENT 'freelancer info'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\\t'
STORED AS TEXTFILE
LOCATION '/export_hive/info';
"""


In [143]:
print pre,
n=0
for i in df_save.columns[:-1]:
    n+=1
    print str(n)+'_'+i.replace('.','_'), \
    str(df_save[i].dtypes).replace('object','STRING').replace('int64','INT').replace('float64','FLOAT')\
    .replace('bool','BOOLEAN')+','
print "26_registration_date DATE",
print post


CREATE EXTERNAL TABLE IF NOT EXISTS info(
1_label STRING,
2_user_id INT,
3_about_company STRING,
4_about_hourly_rate STRING,
5_about_profile_description STRING,
6_about_recommendations INT,
7_country_name STRING,
8_timezone STRING,
9_skills_LEN INT,
10_resume_experience_LEN INT,
11_resume_education_LEN INT,
12_resume_certifications_LEN INT,
13_job_history_job_counts_LEN INT,
14_phone_verified BOOLEAN,
15_payment_verified BOOLEAN,
16_facebook_connected BOOLEAN,
17_summary_reviews INT,
18_summary_on_time STRING,
19_summary_on_budget STRING,
20_entire_history_incomplete INT,
21_entire_history_reviews INT,
22_earnings_score INT,
23_about_display_name STRING,
24_about_public_name STRING,
25_public_firstname STRING,
26_registration_date DATE 
)
COMMENT 'freelancer info'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
LOCATION '/export_hive/info';



### Create ORC table

In [136]:
print """
CREATE TABLE IF NOT EXISTS infoORC(
1_label STRING,
2_user_id INT,
3_about_company STRING,
4_about_hourly_rate STRING,
5_about_profile_description STRING,
6_about_recommendations INT,
7_country_name STRING,
8_timezone STRING,
9_skills_LEN INT,
10_resume_experience_LEN INT,
11_resume_education_LEN INT,
12_resume_certifications_LEN INT,
13_job_history_job_counts_LEN INT,
14_phone_verified BOOLEAN,
15_payment_verified BOOLEAN,
16_facebook_connected BOOLEAN,
17_summary_reviews INT,
18_summary_on_time STRING,
19_summary_on_budget STRING,
20_entire_history_incomplete INT,
21_entire_history_reviews INT,
22_earnings_score INT,
23_about_display_name STRING,
24_about_public_name STRING,
25_public_firstname STRING,
26_registration_date DATE
)
COMMENT 'freelancer info ORC'
STORED AS ORC;
"""


CREATE TABLE IF NOT EXISTS infoORC(
1_label STRING,
2_user_id INT,
3_about_company STRING,
4_about_hourly_rate STRING,
5_about_profile_description STRING,
6_about_recommendations INT,
7_country_name STRING,
8_timezone STRING,
9_skills_LEN INT,
10_resume_experience_LEN INT,
11_resume_education_LEN INT,
12_resume_certifications_LEN INT,
13_job_history_job_counts_LEN INT,
14_phone_verified BOOLEAN,
15_payment_verified BOOLEAN,
16_facebook_connected BOOLEAN,
17_summary_reviews INT,
18_summary_on_time STRING,
19_summary_on_budget STRING,
20_entire_history_incomplete INT,
21_entire_history_reviews INT,
22_earnings_score INT,
23_about_display_name STRING,
24_about_public_name STRING,
25_public_firstname STRING,
26_registration_date DATE
)
COMMENT 'freelancer info ORC'
STORED AS ORC;



### insert data

In [137]:
print """INSERT OVERWRITE TABLE infoorc SELECT * FROM info;"""

INSERT OVERWRITE TABLE infoorc SELECT * FROM info;


In [144]:
df_save.shape

(102514, 26)

In [146]:
%%time
df_save.to_csv("info.hive.line.terminator.csv", header=None, sep='\t', index=False)

CPU times: user 832 ms, sys: 21 ms, total: 853 ms
Wall time: 855 ms


In [145]:
df_save.head()

,label,user_id,about.company,about.hourly_rate,about.profile_description,about.recommendations,country.name,timezone,skills_LEN,resume.experience_LEN,...,summary.reviews,summary.on_time,summary.on_budget,entire_history.incomplete,entire_history.reviews,earnings_score,about.display_name,about.public_name,public_firstname,registration_date
0,Male,1018449,itsol,60,NaN,0,United Kingdom,Europe/London,15,1,...,0,NaN,NaN,0,0,0,techit99,techit99,techit99,2009-03-24
1,Female,8288991,None,20,We've started our professional work in field o...,3,Norway,Europe/Oslo,8,0,...,43,100%,100%,4,43,6,triangle2000,Mahsa I.,Mahsa,2013-07-24
2,Male,314218,goldengatemedia,None,NaN,0,United States,America/Los_Angeles,0,0,...,0,NaN,NaN,0,0,0,webcase,webcase,webcase,2006-12-06
3,Female,1240449,None,None,NaN,0,United Kingdom,Europe/London,3,0,...,0,NaN,NaN,0,0,0,tiabil,tiabil,tiabil,2009-10-12
4,Male,2157025,None,None,I am looking forward to being hired by you. I ...,0,Romania,Europe/Bucharest,20,0,...,0,NaN,NaN,0,0,0,radumihai,Mihai T.,Mihai,2011-02-04


### skills to hive create table csv

In [149]:
query= """
CREATE EXTERNAL TABLE IF NOT EXISTS skills(1_userid INT,
2_AntennaServices INT,
3_Roofing INT,
4_Git INT,
5_Angularjs INT,
6_PhotoRetouching INT,
7_Delphi INT,
8_Poetry INT,
9_SEOAuditing INT,
10_VehicleSignage INT,
11_3dsMax INT,
12_ElectronicForms INT,
13_Flask INT,
14_AdobePremierePro INT,
15_BlogInstall INT,
16_PatternMatching INT,
17_EventPlanning INT,
18_PersonalDevelopment INT,
19_BusinessCards INT,
20_EducationTutoring INT,
21_Norwegian INT,
22_Decking INT,
23_Javascript INT,
24_Moodle INT,
25_Entrepreneurship INT,
26_SoundDesign INT,
27_CookingBaking INT,
28_Lighting INT,
29_CProgramming INT,
30_GeneralOffice INT,
31_Apache INT,
32_Energy INT,
33_edX INT,
34_Freelance INT,
35_OculusRift INT,
36_WIKI INT,
37_Training INT,
38_PetSitting INT,
39_Plugin INT,
40_NoSQLCouchMongo INT,
41_Django INT,
42_InvestmentResearch INT,
43_DataWarehousing INT,
44_SocialMediaMarketing INT,
45_CommercialCleaning INT,
46_Windows8 INT,
47_QuickBase INT,
48_TechnicalWriting INT,
49_TelecommunicationsEngineering INT,
50_HBase INT,
51_SocialEngine INT,
52_GraphicsProgramming INT,
53_PhoneSupport INT,
54_Jewellery INT,
55_Fortran INT,
56_Tax INT,
57_BuildingDesigner INT,
58_COBOL INT,
59_Yiddish INT,
60_GoogleCardboard INT,
61_EssayWriting INT,
62_Jabber INT,
63_DataEntry INT,
64_KeywordResearch INT,
65_LaTeX INT,
66_CVLibrary INT,
67_Chordiant INT,
68_MicrosoftOutlook INT,
69_ParallelProcessing INT,
70_Flash3D INT,
71_JavaFX INT,
72_TattooDesign INT,
73_3DModelMaker INT,
74_Housework INT,
75_Storyboard INT,
76_ECMAScript INT,
77_NintexWorkflow INT,
78_CMS INT,
79_DotNetNuke INT,
80_Delivery INT,
81_AMQP INT,
82_KineticTypography INT,
83_GeneticEngineering INT,
84_Automotive INT,
85_MagicLeap INT,
86_InteriorDesign INT,
87_WebScraping INT,
88_HTML INT,
89_Pentaho INT,
90_Imaging INT,
91_ForumSoftware INT,
92_AppleiBooksAuthor INT,
93_SalesforceAppDevelopment INT,
94_Swift INT,
95_CSCart INT,
96_FashionModeling INT,
97_FlashAnimation INT,
98_LinearProgramming INT,
99_Documentation INT,
100_Typography INT,
101_ReportWriting INT,
102_PostProduction INT,
103_GameDesign INT,
104_WebsiteAnalytics INT,
105_Latvian INT,
106_Resumes INT,
107_Azure INT,
108_RegularExpressions INT,
109_StickerDesign INT,
110_SocialNetworking INT,
111_Vuforia INT,
112_ProjectManagement INT,
113_Croatian INT,
114_TYPO3 INT,
115_RProgrammingLanguage INT,
116_RWD INT,
117_Fundraising INT,
118_FinancialResearch INT,
119_RemoteSensing INT,
120_Excavation INT,
121_CloudComputing INT,
122_TimeManagement INT,
123_SignDesign INT,
124_ProductManagement INT,
125_Periscope INT,
126_VisaImmigration INT,
127_CubeCart INT,
128_UserExperienceDesign INT,
129_Arabic INT,
130_OpenVMS INT,
131_FinancialMarkets INT,
132_Ning INT,
133_Codeigniter INT,
134_Yii INT,
135_Wufoo INT,
136_FullStackDevelopment INT,
137_SearchEngineMarketing INT,
138_BinaryAnalysis INT,
139_HairStyles INT,
140_Gamification INT,
141_GoogleChrome INT,
142_Hindi INT,
143_WebSecurity INT,
144_DOS INT,
145_InternetSecurity INT,
146_GIMP INT,
147_webMethods INT,
148_ComputerGraphics INT,
149_MechanicalEngineering INT,
150_PropertyLaw INT,
151_Bosnian INT,
152_Handyman INT,
153_LogoDesign INT,
154_PetroleumEngineering INT,
155_VideoScribe INT,
156_UserInterfaceDesign INT,
157_JSP INT,
158_ColdFusion INT,
159_Fencing INT,
160_Russian INT,
161_BambooFlooring INT,
162_AppDeveloper INT,
163_Bengali INT,
164_PLCSCADA INT,
165_eLearning INT,
166_PayPalAPI INT,
167_PublicRelations INT,
168_360degreevideo INT,
169_ExpressionEngine INT,
170_CoversPackaging INT,
171_MVC INT,
172_Clothesline INT,
173_HotWater INT,
174_LandscapingGardening INT,
175_PhotoshopDesign INT,
176_Painting INT,
177_Powerpoint INT,
178_MotionGraphics INT,
179_cURL INT,
180_German INT,
181_BusinessCoaching INT,
182_GreaseMonkey INT,
183_GoogleAnalytics INT,
184_vBulletin INT,
185_CopyTyping INT,
186_Basque INT,
187_BroadcastEngineering INT,
188_Wireframes INT,
189_ViralMarketing INT,
190_ERP INT,
191_Ceilings INT,
192_Vietnamese INT,
193_WindowsPhone INT,
194_Dutch INT,
195_OculusMobileSDK INT,
196_PackingShipping INT,
197_RTOS INT,
198_SwingJava INT,
199_Infographics INT,
200_Poet INT,
201_MYOB INT,
202_Erlang INT,
203_Christmas INT,
204_GoogleMapsAPI INT,
205_Grails INT,
206_Typescript INT,
207_Balustrading INT,
208_ShopifyTemplates INT,
209_Titanium INT,
210_Welsh INT,
211_Marketing INT,
212_MedicalWriting INT,
213_AeronauticalEngineering INT,
214_LearningManagementSystemsLMS INT,
215_PPCMarketing INT,
216_WebCrawling INT,
217_CUDA INT,
218_VirtualMachines INT,
219_Spark INT,
220_OrganizationalChangeManagement INT,
221_SmartyPHP INT,
222_Albanian INT,
223_Troubleshooting INT,
224_IBMTivoli INT,
225_Xojo INT,
226_FPGA INT,
227_OpenVPN INT,
228_Lisp INT,
229_SpeechWriting INT,
230_Bootstrap INT,
231_Hive INT,
232_CLIPS INT,
233_Plumbing INT,
234_Visualization INT,
235_Blackberry INT,
236_BeautifulSoup INT,
237_OAuth INT,
238_AdobeFlash INT,
239_Guttering INT,
240_CSS3 INT,
241_Geolocation INT,
242_Solaris INT,
243_3DModelling INT,
244_LogisticsShipping INT,
245_PSDtoHTML INT,
246_OrderProcessing INT,
247_MySpace INT,
248_Japanese INT,
249_Umbraco INT,
250_MachineLearning INT,
251_Decoration INT,
252_Haskell INT,
253_WordPress INT,
254_TechnicalSupport INT,
255_Linux INT,
256_Asphalt INT,
257_Lithuanian INT,
258_AppleMotion INT,
259_BlogWriting INT,
260_InstagramAPI INT,
261_J2EE INT,
262_PEGAPRPC INT,
263_Telugu INT,
264_Brackets INT,
265_IntuitQuickBooks INT,
266_Microbiology INT,
267_LINQ INT,
268_OpenSSL INT,
269_Airbnb INT,
270_MuralPainting INT,
271_GooglePlus INT,
272_SenchaYahooUI INT,
273_BookWriting INT,
274_FinalCutPro INT,
275_Makerbot INT,
276_QlikView INT,
277_DataScraping INT,
278_CoatingMaterials INT,
279_EngineeringDrawing INT,
280_Nginx INT,
281_BuildingSurveyors INT,
282_TestingQA INT,
283_GoogleSketchUp INT,
284_Heroku INT,
285_SAS INT,
286_SAP INT,
287_ShortStories INT,
288_SamsungAccessorySDK INT,
289_PhotoshopCoding INT,
290_Illustrator INT,
291_Kotlin INT,
292_ScriptInstall INT,
293_Compliance INT,
294_GarageBand INT,
295_Joomla INT,
296_GoogleAppEngine INT,
297_SymfonyPHP INT,
298_iPhone INT,
299_AdobeMuse INT,
300_BuildingCertifiers INT,
301_Writing INT,
302_MariaDB INT,
303_Dari INT,
304_Eclipse INT,
305_Mailwizz INT,
306_PosterDesign INT,
307_iMovie INT,
308_CircuitDesign INT,
309_Locksmith INT,
310_Recruitment INT,
311_API INT,
312_CreativeWriting INT,
313_SystemAdmin INT,
314_InternetResearch INT,
315_SocialMediaManagement INT,
316_AdPlanningBuying INT,
317_Statistics INT,
318_FreelancerAPI INT,
319_PhotoRestoration INT,
320_Transcription INT,
321_Interspire INT,
322_Slovenian INT,
323_WindowsMobile INT,
324_Legal INT,
325_Perl INT,
326_SocketIO INT,
327_Lua INT,
328_BusinessIntelligence INT,
329_Inspections INT,
330_Vuejs INT,
331_DataProcessing INT,
332_Print INT,
333_Weebly INT,
334_OCR INT,
335_Czech INT,
336_CRELoaded INT,
337_HumanSciences INT,
338_Mailchimp INT,
339_BackendDevelopment INT,
340_WordProcessing INT,
341_BookArtist INT,
342_CatchPhrases INT,
343_MicrosoftAccess INT,
344_Cinematography INT,
345_Medical INT,
346_ShoppingCarts INT,
347_CreativeDesign INT,
348_CorporateIdentity INT,
349_Word INT,
350_Samsung INT,
351_iBeacon INT,
352_VBScript INT,
353_VideoProduction INT,
354_Scala INT,
355_Presentations INT,
356_HouseCleaning INT,
357_EnglishSpelling INT,
358_PatternMaking INT,
359_OSCommerce INT,
360_ComputerSecurity INT,
361_AsbestosRemoval INT,
362_ProductSourcing INT,
363_Drawing INT,
364_CSS INT,
365_AdvertisementDesign INT,
366_SoftwareDevelopment INT,
367_SoftwareArchitecture INT,
368_FinancialAnalysis INT,
369_BigData INT,
370_ASPNET INT,
371_SoftwareTesting INT,
372_Tumblr INT,
373_Virtuemart INT,
374_Electricians INT,
375_ADONET INT,
376_ClimateSciences INT,
377_RealEstate INT,
378_Interpreter INT,
379_Demolition INT,
380_AmazonWebServices INT,
381_ePub INT,
382_MarketingStrategy INT,
383_RedHat INT,
384_Copyright INT,
385_Arduino INT,
386_GameConsoles INT,
387_OpenCL INT,
388_Workshops INT,
389_OpenCV INT,
390_Pickup INT,
391_Prezi INT,
392_FacebookMarketing INT,
393_Grunt INT,
394_2DAnimation INT,
395_AfterEffects INT,
396_UMLDesign INT,
397_eLearningDesigner INT,
398_HomeAutomation INT,
399_CorelPainter INT,
400_ASP INT,
401_BuildingArchitecture INT,
402_AmazonKindle INT,
403_Contracts INT,
404_CartographyMaps INT,
405_ExpressJS INT,
406_DigitalDesign INT,
407_MapReduce INT,
408_Catalan INT,
409_BulkMarketing INT,
410_WebServices INT,
411_Odoo INT,
412_SEO INT,
413_Bower INT,
414_WPF INT,
415_Debian INT,
416_Interiors INT,
417_Genealogy INT,
418_IonicFramework INT,
419_Solidworks INT,
420_BrandManagement INT,
421_IBMBPM INT,
422_Tableau INT,
423_DatabaseDevelopment INT,
424_Silverlight INT,
425_StatisticalAnalysis INT,
426_Oracle INT,
427_PDF INT,
428_MatlabandMathematica INT,
429_VerylargescaleintegrationVLSI INT,
430_Scripting INT,
431_History INT,
432_SupplierSourcing INT,
433_Flyscreens INT,
434_ITIL INT,
435_AffiliateMarketing INT,
436_TeklaStructures INT,
437_LeapMotionSDK INT,
438_ObjectiveC INT,
439_Preproduction INT,
440_VisualBasicforApps INT,
441_ApacheSolr INT,
442_AndroidWearSDK INT,
443_ResearchWriting INT,
444_MMORPG INT,
445_Cassandra INT,
446_ScikitLearn INT,
447_GameSalad INT,
448_3DAnimation INT,
449_AdobeFireworks INT,
450_BoonexDolphin INT,
451_SugarCRM INT,
452_BPO INT,
453_LifeCoaching INT,
454_Subversion INT,
455_Animation INT,
456_ProductDescriptions INT,
457_ValuationAppraisal INT,
458_Balsamiq INT,
459_LandscapeDesign INT,
460_Flex INT,
461_Moving INT,
462_Ukrainian INT,
463_ShellScript INT,
464_TestAutomation INT,
465_FashionDesign INT,
466_BrandMarketing INT,
467_Slogans INT,
468_Maltese INT,
469_MicrosoftOffice INT,
470_TShirts INT,
471_NaturalLanguage INT,
472_Sculpturing INT,
473_Qt INT,
474_AwsLambda INT,
475_Hebrew INT,
476_TSQLTransactStructuresQueryLanguage INT,
477_MobileAppTesting INT,
478_MLM INT,
479_QuarkXPress INT,
480_Communications INT,
481_Pavement INT,
482_Piping INT,
483_FlyerDesign INT,
484_Advertising INT,
485_MacOS INT,
486_CallCenter INT,
487_VisualFoxpro INT,
488_Bookkeeping INT,
489_ScrumDevelopment INT,
490_3DRendering INT,
491_StructuralEngineering INT,
492_AudioProduction INT,
493_Photoshop INT,
494_Attorney INT,
495_CopyEditing INT,
496_CallControlXML INT,
497_HTCVive INT,
498_AppceleratorTitanium INT,
499_ExcelMacros INT,
500_InstagramMarketing INT,
501_MarketResearch INT,
502_EnglishUK INT,
503_AugmentedReality INT,
504_FiniteElementAnalysis INT,
505_ScientificResearch INT,
506_Accounting INT,
507_Unit4BusinessWorld INT,
508_Tiling INT,
509_Scheme INT,
510_Protoshare INT,
511_WebSearch INT,
512_NeuralNetworks INT,
513_OpenGL INT,
514_LabelDesign INT,
515_HumanResources INT,
516_Hadoop INT,
517_Plesk INT,
518_Research INT,
519_DatabaseAdministration INT,
520_FreeSwitch INT,
521_VideoBroadcasting INT,
522_IBMBluemix INT,
523_MySQL INT,
524_IBMWebsphereTransformationTool INT,
525_J2ME INT,
526_Drafting INT,
527_AirConditioning INT,
528_SEOWriting INT,
529_Carpentry INT,
530_Moz INT,
531_Bulgarian INT,
532_OpenJournalSystems INT,
533_Indonesian INT,
534_Photography INT,
535_RocketEngine INT,
536_RubyonRails INT,
537_French INT,
538_Salesforcecom INT,
539_Slovakian INT,
540_Stripe INT,
541_LinnworksOrderManagement INT,
542_Translation INT,
543_Virtuozzo INT,
544_IconDesign INT,
545_Videography INT,
546_JavaSpring INT,
547_BusinessAnalysis INT,
548_Finnish INT,
549_AdobeLiveCycleDesigner INT,
550_MicrosoftExchange INT,
551_WindowsServer INT,
552_Biztalk INT,
553_Yarn INT,
554_ComputerHelp INT,
555_Reviews INT,
556_ProductDesign INT,
557_Serbian INT,
558_XHTML INT,
559_Veeam INT,
560_ImageProcessing INT,
561_Cinema4D INT,
562_Concreting INT,
563_Italian INT,
564_FrontendDevelopment INT,
565_AutoCAD INT,
566_Instagram INT,
567_Coding INT,
568_TaxLaw INT,
569_ConceptArt INT,
570_Proofreading INT,
571_Magento INT,
572_DataAnalytics INT,
573_Urdu INT,
574_UserInterfaceIA INT,
575_VisualBasic INT,
576_PHP INT,
577_GoogleAdwords INT,
578_iPad INT,
579_Instrumentation INT,
580_Flashmob INT,
581_nodejs INT,
582_IIS INT,
583_Wikipedia INT,
584_Sketching INT,
585_eBay INT,
586_Kinect INT,
587_CasperJS INT,
588_FurnitureAssembly INT,
589_ParallelsAutomation INT,
590_Android INT,
591_PackageDesign INT,
592_StorageAreaNetworks INT,
593_CRM INT,
594_Biology INT,
595_Ghostwriting INT,
596_Shopping INT,
597_backbonejs INT,
598_Analytics INT,
599_AdobeAir INT,
600_CarpetRepairLaying INT,
601_Scrum INT,
602_ExtensionsAdditions INT,
603_Volusion INT,
604_Symbian INT,
605_NetworkAdministration INT,
606_PressReleases INT,
607_AmazonFire INT,
608_Minitab INT,
609_VerilogVHDL INT,
610_VisualMerchandising INT,
611_Xero INT,
612_Nanotechnology INT,
613_Afrikaans INT,
614_AppleSafari INT,
615_MicrosoftSQLServer INT,
616_PortugueseBrazil INT,
617_Kitchen INT,
618_Xamarin INT,
619_Econometrics INT,
620_SeleniumWebdriver INT,
621_VPS INT,
622_DDS INT,
623_XAML INT,
624_Drones INT,
625_Programming INT,
626_LegalResearch INT,
627_Procurement INT,
628_UNIX INT,
629_Geospatial INT,
630_PenetrationTesting INT,
631_REALbasic INT,
632_Powershell INT,
633_VoiceXML INT,
634_PCBLayout INT,
635_Firefox INT,
636_AnythingGoes INT,
637_ArticleWriting INT,
638_IndustrialEngineering INT,
639_ArtsCrafts INT,
640_CaricatureCartoons INT,
641_LabVIEW INT,
642_GoogleWebsiteOptimizer INT,
643_Leads INT,
644_Design INT,
645_FlowCharts INT,
646_Disposals INT,
647_Dynamics INT,
648_Axure INT,
649_ForumPosting INT,
650_GeotechnicalEngineering INT,
651_ISO9001 INT,
652_Shopify INT,
653_Turkish INT,
654_VoiceTalent INT,
655_Zendesk INT,
656_Fiction INT,
657_FormatLayout INT,
658_Prestashop INT,
659_Carports INT,
660_Music INT,
661_jQueryPrototype INT,
662_BuildingConsultants INT,
663_UsabilityTesting INT,
664_OpenStack INT,
665_Dating INT,
666_Columns INT,
667_AdobeIllustrator INT,
668_LawnMowing INT,
669_CementBondingAgents INT,
670_WindowsAPI INT,
671_Screenwriting INT,
672_Sewing INT,
673_phpMyAdmin INT,
674_Malay INT,
675_HomeDesign INT,
676_AdobePagemaker INT,
677_JSON INT,
678_NintexForms INT,
679_VMware INT,
680_BusinessCatalyst INT,
681_InventoryManagement INT,
682_Editing INT,
683_VideoEditing INT,
684_DampProofing INT,
685_VoIP INT,
686_LotusNotes INT,
687_WindowsDesktop INT,
688_CATIA INT,
689_ParallelsDesktop INT,
690_XSLT INT,
691_DataScience INT,
692_Splunk INT,
693_AdobeDreamweaver INT,
694_CADCAM INT,
695_QualtricsSurveyPlatform INT,
696_Courses INT,
697_ChemicalEngineering INT,
698_TwitterAPI INT,
699_AdobeInDesign INT,
700_ConstructionMonitoring INT,
701_MaterialsEngineering INT,
702_PSD2CMS INT,
703_Debugging INT,
704_Gardening INT,
705_GameDevelopment INT,
706_Health INT,
707_Tamil INT,
708_TraditionalChineseTaiwan INT,
709_Typing INT,
710_Sales INT,
711_Snapchat INT,
712_Kannada INT,
713_RiskManagement INT,
714_BluetoothLowEnergyBLE INT,
715_PropertyManagement INT,
716_FrenchCanadian INT,
717_Quantum INT,
718_PhotoEditing INT,
719_GeographicalInformationSystemGIS INT,
720_Building INT,
721_Copywriting INT,
722_BashScripting INT,
723_MODx INT,
724_XMPP INT,
725_Helpdesk INT,
726_Redis INT,
727_BrochureDesign INT,
728_PhpNuke INT,
729_EmailMarketing INT,
730_Astrophysics INT,
731_Installation INT,
732_Docker INT,
733_RenewableEnergyDesign INT,
734_Filipino INT,
735_Python INT,
736_EnglishUS INT,
737_XXX INT,
738_ArticleSubmission INT,
739_SteamAPI INT,
740_ProposalBidWriting INT,
741_ApacheAnt INT,
742_Knockoutjs INT,
743_Palm INT,
744_CookingRecipes INT,
745_Etsy INT,
746_eCommerce INT,
747_eBooks INT,
748_TextileEngineering INT,
749_GPGPU INT,
750_CaptureNX2 INT,
751_GoogleWebmasterTools INT,
752_CleanTechnology INT,
753_ManufacturingDesign INT,
754_SQLite INT,
755_AsteriskPBX INT,
756_OpenBravo INT,
757_AutodeskRevit INT,
758_InvitationDesign INT,
759_HTML5 INT,
760_MakeUp INT,
761_VoiceArtist INT,
762_Biotechnology INT,
763_PICKMultivalueDB INT,
764_Physics INT,
765_BSD INT,
766_WebsiteTesting INT,
767_VisualArts INT,
768_Squarespace INT,
769_SurfboardDesign INT,
770_FramesTrusses INT,
771_PhoneGap INT,
772_Branding INT,
773_Zend INT,
774_Golang INT,
775_FloorCoatings INT,
776_OnlineWriting INT,
777_CrystalReports INT,
778_BannerDesign INT,
779_AS400iSeries INT,
780_GoogleCloudStorage INT,
781_GeneralLabor INT,
782_DigitalMarketing INT,
783_Estonian INT,
784_DataExtraction INT,
785_Zoho INT,
786_3DPrinting INT,
787_Bitcoin INT,
788_AdobeCaptivate INT,
789_Scrapy INT,
790_WooCommerce INT,
791_BMCRemedy INT,
792_Dthreejs INT,
793_Alibaba INT,
794_DatabaseProgramming INT,
795_WebsiteDesign INT,
796_AppleWatch INT,
797_iOSDevelopment INT,
798_FoodTakeaway INT,
799_PropertyDevelopment INT,
800_Nokia INT,
801_YouTube INT,
802_ExcelVBA INT,
803_FacebookAPI INT,
804_GoPro INT,
805_RaspberryPi INT,
806_ComputerScience INT,
807_Thai INT,
808_Carwashing INT,
809_EnglishGrammar INT,
810_OpenCart INT,
811_DataLifeEngine INT,
812_ArgusMonitoringSoftware INT,
813_AgileDevelopment INT,
814_FileMaker INT,
815_Drupal INT,
816_SVG INT,
817_Greek INT,
818_Zbrush INT,
819_StationeryDesign INT,
820_MiningEngineering INT,
821_Reactjs INT,
822_ProjectScheduling INT,
823_AJAX INT,
824_WHMCS INT,
825_DataMining INT,
826_Malayalam INT,
827_CakePHP INT,
828_VideoUpload INT,
829_Millwork INT,
830_Cisco INT,
831_OpenVZ INT,
832_Redshift INT,
833_TaoBaoAPI INT,
834_Siebel INT,
835_Newsletters INT,
836_BusinessPlans INT,
837_GoogleWebToolkit INT,
838_ContentStrategy INT,
839_LaundryandIroning INT,
840_MQTT INT,
841_CleaningDomestic INT,
842_SQL INT,
843_BigCommerce INT,
844_ElectricalEngineering INT,
845_Camtasia INT,
846_AutodeskInventor INT,
847_TestStand INT,
848_Swedish INT,
849_Laravel INT,
850_Management INT,
851_Java INT,
852_Pinterest INT,
853_IndustrialDesign INT,
854_Commercials INT,
855_JDF INT,
856_Excel INT,
857_SquidCache INT,
858_TallyDefinitionLanguage INT,
859_ApplianceRepair INT,
860_XML INT,
861_FinancialPlanning INT,
862_ContentWriting INT,
863_CleaningCarpet INT,
864_BusinessCardDesign INT,
865_IKEAInstallation INT,
866_Wolfram INT,
867_VirtualAssistant INT,
868_Danish INT,
869_VarnishCache INT,
870_Cryptography INT,
871_Emberjs INT,
872_CorelDraw INT,
873_DesktopSupport INT,
874_ObjectOrientedProgrammingOOP INT,
875_PeopleSoft INT,
876_phpFox INT,
877_MeteorJS INT,
878_HomeOrganization INT,
879_Ruby INT,
880_ApplianceInstallation INT,
881_MobileAppDevelopment INT,
882_CocoaTouch INT,
883_AutoHotkey INT,
884_Algorithm INT,
885_CustomerService INT,
886_Punjabi INT,
887_Economics INT,
888_LandingPages INT,
889_WebOS INT,
890_WebHosting INT,
891_VertexFX INT,
892_WindowsCE INT,
893_EditorialWriting INT,
894_Polish INT,
895_Assembly INT,
896_GrowthHacking INT,
897_Microsoft INT,
898_Prolog INT,
899_Hungarian INT,
900_ChromeOS INT,
901_Geology INT,
902_Cocoa INT,
903_CivilEngineering INT,
904_SalesPromotion INT,
905_CustomerSupport INT,
906_ChefConfigurationManagement INT,
907_Microstation INT,
908_TravelWriting INT,
909_Telemarketing INT,
910_Journalist INT,
911_Ethereum INT,
912_YahooStoreDesign INT,
913_Psychology INT,
914_GPS INT,
915_Sharepoint INT,
916_BrainStorming INT,
917_vTiger INT,
918_AdobeLightroom INT,
919_VideoServices INT,
920_MicrosoftVisio INT,
921_Unity3D INT,
922_QualitativeResearch INT,
923_Korean INT,
924_x86x64Assembler INT,
925_ContentMarketing INT,
926_PaymentGatewayIntegration INT,
927_CGI INT,
928_Nutrition INT,
929_PestControl INT,
930_Finance INT,
931_4D INT,
932_Microcontroller INT,
933_Autotask INT,
934_Illustration INT,
935_SimplifiedChineseChina INT,
936_ConceptDesign INT,
937_Landscaping INT,
938_AppDesigner INT,
939_Patents INT,
940_Ubuntu INT,
941_GrantWriting INT,
942_YardWorkRemoval INT,
943_WebDevelopment INT,
944_HeatingSystems INT,
945_HPOpenview INT,
946_Sphinx INT,
947_Portuguese INT,
948_FurnitureDesign INT,
949_Blockchain INT,
950_WebsiteManagement INT,
951_GraphicDesign INT,
952_Metatrader INT,
953_Mechatronics INT,
954_LinkBuilding INT,
955_DNS INT,
956_CleaningUpholstery INT,
957_Sports INT,
958_EmbeddedSoftware INT,
959_WatchKit INT,
960_ZenCart INT,
961_BlogDesign INT,
962_AudioServices INT,
963_RESTful INT,
964_Twitter INT,
965_EmailHandling INT,
966_SpanishSpain INT,
967_EventStaffing INT,
968_Macedonian INT,
969_Engineering INT,
970_Puppet INT,
971_BusinessWriting INT,
972_ActionScript INT,
973_Startups INT,
974_Insurance INT,
975_ClassifiedsPosting INT,
976_Bathroom INT,
977_TraditionalChineseHongKong INT,
978_Publishing INT,
979_Flooring INT,
980_Romanian INT,
981_Applescript INT,
982_NET INT,
983_3DDesign INT,
984_BuyerSourcing INT,
985_Blog INT,
986_TwitterMarketing INT,
987_SPSSStatistics INT,
988_Mathematics INT,
989_Electronics INT,
990_Templates INT,
991_FaceRecognition INT,
992_Maya INT,
993_InternetMarketing INT,
994_Vectorization INT,
995_AppleCompressor INT,
996_Payroll INT,
997_TelephoneHandling INT,
998_Dart INT,
999_Wireless INT,
1000_FinaleSibelius INT,
1001_XQuery INT,
1002_CProgramming INT,
1003_LegalWriting INT,
1004_Manufacturing INT,
1005_GoogleAdsense INT,
1006_AerospaceEngineering INT,
1007_Audit INT,
1008_EmailDeveloper INT,
1009_Robotics INT,
1010_EmploymentLaw INT,
1011_ConversionRateOptimisation INT,
1012_Metro INT,
1013_ActiveDirectory INT,
1014_VBNET INT,
1015_Filmmaking INT,
1016_AcademicWriting INT,
1017_Elasticsearch INT,
1018_ParallaxScrolling INT,
1019_LESSSassSCSS INT,
1020_ArticleRewriting INT,
1021_ApacheMaven INT,
1022_Agronomy INT,
1023_MicrosoftHololens INT,
1024_EquipmentHire INT,
1025_DataCleansing INT,
1026_PostgreSQL INT,
1027_Netbeans INT,
1028_CProgramming INT,
1029_Paytrace INT,
1030_Weddings INT,
1031_AppleLogicPro INT,
1032_Linkedin INT,
1033_MonetDB INT,
1034_AutodeskSketchbookPro INT,
1035_Virtualization INT,
1036_MicrosoftExpression INT,
1037_Spanish INT,
1038_GoogleEarth INT,
1039_VirtualWorlds INT,
1040_ArtificialIntelligence INT,
1041_Wix INT)
COMMENT 'freelancer skills'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/export_hive/skills'
TBLPROPERTIES("skip.header.line.count"="1");
"""

In [151]:
%%time
df_save.to_csv("info.hive.withheader.tab.csv", sep='\t', index=False)

CPU times: user 852 ms, sys: 14 ms, total: 866 ms
Wall time: 863 ms
